<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Software-required" data-toc-modified-id="Software-required-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Software required</a></span></li><li><span><a href="#Download-1000Genomes-data" data-toc-modified-id="Download-1000Genomes-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Download 1000Genomes data</a></span></li><li><span><a href="#Running-LMI" data-toc-modified-id="Running-LMI-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Running LMI</a></span><ul class="toc-item"><li><span><a href="#Download-data-and-scripts" data-toc-modified-id="Download-data-and-scripts-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Download data and scripts</a></span></li><li><span><a href="#Format-sumstats-file:" data-toc-modified-id="Format-sumstats-file:-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Format sumstats file:</a></span></li><li><span><a href="#Now-run-the-script-this-like-this." data-toc-modified-id="Now-run-the-script-this-like-this.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Now run the script this like this.</a></span></li></ul></li><li><span><a href="#Format-&amp;-prepare-GWAS-sumstats-&amp;-LMI-results" data-toc-modified-id="Format-&amp;-prepare-GWAS-sumstats-&amp;-LMI-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Format &amp; prepare GWAS sumstats &amp; LMI results</a></span></li></ul></div>

### Software required

Download and install the following required software and add them to your path.

Plink: https://www.cog-genomics.org/plink/1.9/

Vcftools http://vcftools.sourceforge.net/

R https://cran.r-project.org/

### Download 1000Genomes data
You can use any VCF from any population, but our study was run with CEU population (n=85) (file `CEU_indv.txt`, for a list of the individuals) and MAF < 0.01.

See below for the example VCF set we used in our study.

### Running LMI

#### Download data and scripts
Download the gzipped folder from: https://github.com/pollicipes/Local-Moran-Index-1D/
Move to that folder and you'll find some scripts and one folder named `data` containing the sumstats and the CEU sample names used.
To calculate LMI You only have to run the main script `run_LMI.sh` following the commands below.

In the folder you just downloaded, go to `/data/1000genomes_CEU_MAF001.tar.gz`, where we have provided the VCF set we used for the analysis. 

Unzip the folder like this:

In [ ]:
tar xvzf /data/1000genomes_CEU_MAF001.tar.gz

#### Format sumstats file:
The order of the columns for the sumstats **MUST** be: (not need to have the same string headers, but columns must contain what the header refers to)

`SNP	CHR	BP	OR	P`

Being respectively: SNP name code, chromosome, base pair position, effect size, pvalue

In [ ]:
# Switching to the folder you just download
cd ${YOUR_PATH}/Local-Moran-Index-1D/

#### Now run the script this like this.
Set the variables and the options you prefer. This code runs for a single chromosome. You can parallelize the code in several processors; one chromosome each.

In [ ]:
./run_lmi.sh 
    ${YOUR_PATH}/data/                           # Starting folder full path
    ${YOUR_PATH}/data/chr_files_1KG_CEU_MAF001/  # 1000 genomes files full path
    sumstats.raw.pangen_isblac_epicuro           # Summary statistics (should be on the starting folder)
    22                                           # Chromosome to process
    500000                                       # Window on which calculate LMI
    0.05                                         # difference allowed in MAF
    alias                                        # alias to identify the sample.                                                 

### Format & prepare GWAS sumstats & LMI results

In our sumstats we have several cohorts and the resulting pvalues and effect sizes for each one of them. We selected the `pangen_isblac_epicuro` cohort.

In [5]:
# working folder
wd=${YOUR_PATH}/data;
cd ${wd};

In [6]:
# Substitute header to suitable format:
# From the full raw sumstats, select only pangen-isblac epicuro 
cut -f1-4,6 sumstats.raw | sed 1d > sumstats.raw.pangen_isblac_epicuro
# Add header manually.

In [7]:
# how many markers with pv <1e-4 in this dataset
awk '$5 < 0.0001' sumstats.raw.pangen_isblac_epicuro| wc -l

89


Concatenate all LMI values for each chromosome

In [8]:
cat results_LMI/sumstats.raw.pangen_isblac_epicuro_localMoranI.chr* > results_LMI/sumstats.raw.pangen_isblac_epicuro_localMoranI.all

Proceed to `01_GWAS_LMI_integration`, using `R`

**END**